## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

c:\Users\natha\Desktop\FACENS\TCC\ai-llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
from typing import List

def load_documents(base_path: str, documents: List, text_loader_kwargs: dict = None):
    """
    Carrega arquivos .md de um caminho base. Usa DirectoryLoader para pastas com subdiretórios
    e TextLoader para pastas com apenas arquivos .md.

    Args:
        base_path (str): Caminho onde estão os arquivos ou subpastas.
        documents (List): Lista onde os documentos serão adicionados.
        text_loader_kwargs (dict, optional): Opções para o TextLoader.
    """
    if text_loader_kwargs is None:
        text_loader_kwargs = {'encoding': 'utf-8'}

    # Verifica se há subdiretórios
    subfolders = [f for f in glob.glob(os.path.join(base_path, "*")) if os.path.isdir(f)]

    if subfolders:
        # Caso tenha subpastas, usar DirectoryLoader para cada uma
        for folder in subfolders:
            doc_type = os.path.basename(folder)
            print(f"[DirectoryLoader] Carregando de: {doc_type}")
            loader = DirectoryLoader(
                folder,
                glob="**/*.md",
                loader_cls=TextLoader,
                loader_kwargs=text_loader_kwargs
            )
            folder_docs = loader.load()
            for doc in folder_docs:
                doc.metadata["doc_type"] = doc_type
                documents.append(doc)
    else:
        # Caso não tenha subpastas, assumimos que a pasta contém arquivos .md diretamente
        md_files = glob.glob(os.path.join(base_path, "*.md"))
        for file_path in md_files:
            print(f"[TextLoader] Carregando arquivo: {os.path.basename(file_path)}")
            loader = TextLoader(file_path, **text_loader_kwargs)
            doc = loader.load()[0]
            doc.metadata["doc_type"] = os.path.basename(base_path)
            documents.append(doc)


documents = []

load_documents("../preprocessing/knowledge-base", documents)

print(f"Total de documentos carregados: {len(documents)}")      
print(documents)  

[DirectoryLoader] Carregando de: about
[DirectoryLoader] Carregando de: products
Total de documentos carregados: 12
[Document(metadata={'source': '..\\preprocessing\\knowledge-base\\about\\Estratégia de Expansão - GPT.md', 'doc_type': 'about'}, page_content='## 1 . Expansão Geográfica na América do Sul\n\nComo vocês têm exclusividade , faz sentido expandir para outros países . Algumas ideias:\n\n- ● Identificar os mercados mais promissores: Argentina , Chile e Peru têm setores fofortes de mineração e construção . Colômbia e Paraguai também podem ter demanda por reparo de maquinário pesado .\n- ● Criar um canal de vendas internacional: Pode ser um site multilíngue ou um parceiro local para cada país .\n- ● Aproveitar feiras internacionais: Participar de eventos na América do Sul voltados para mineração , construção e manutenção industrial .\n\n## 2 . Reforçar a Geração de Leads e VeVendas Ativas\n\n- ● Inbound Marketing mais agressivo: Criar conteúdos educativos (artigos , vídeos e case

In [4]:
documents[10]

Document(metadata={'source': '..\\preprocessing\\knowledge-base\\products\\S60\\Manual de Operação S60.md', 'doc_type': 'products'}, page_content='# Manual de Operação – Mandrilhadora Portátil S60\n\nGuia prático de uso e segurança para operação da Mandrilhadora S60  \nPreparado por: JVF Máquinas\n\n---\n\n## 1. Centralização da Barra ao Eixo\n\n- Meça o diâmetro do furo da peça (65–600mm).\n- Ajuste a ferramenta de centralização com base nos dados.\n- Insira a ferramenta em cruz em ambos os lados do furo.\n- Realize pontos de solda para fixação.\n- Instale a barra de mandrilhamento e o suporte de ferramenta.\n- Gire a barra, ajuste a altura e verifique a movimentação livre.\n\n---\n\n## 2. Centralização com Ferramenta em "T"\n\n- Meça o furo e ajuste a ferramenta centralizadora.\n- Insira a barra e ajuste a distância ao centro.\n- Aperte os parafusos de fixação.\n- Use relógio comparador magnético para ajuste fino.\n- Repita o ajuste nas duas extremidades.\n\n---\n\n## 3. Instalação d

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1196, which is longer than the specified 1000


In [6]:
len(chunks)

169

In [7]:
chunks[5]

Document(metadata={'source': '..\\preprocessing\\knowledge-base\\about\\Estratégia de Expansão - GPT.md', 'doc_type': 'about'}, page_content='- ● Pipedrive ou Zoho CRM (para organizar contatos e negociações) .\n- ● WhatsApp Business API (para manter comunicação profissional com clientes) .\n\n## SEGUNDOS PAPASSOS (MÉDIO PRAZO)\n\nDepois de fofortalecer a base , podemos avançar para ações mais estratégicas:\n\n## 4 . Expandir a Presença Internacional\n\n## ➡ O que fazer:\n\n- ● Participar de feiras na América do Sul voltadas para mineração , construção e reparo de equipamentos .\n- ● Traduzir materiais de vendas e adaptar a abordagem para cada país .\n- ● Criar parcerias locais (revendedores ou representantes) .\n\n## ➡ TeTecnologias que ajajudam:\n\n- ● ChatGPT para tradução rápida de conteúdos comerciais .\n- ● Google Trends e SEMrush (para entender a demanda por máquinas de solda e embuchamento em cada país) .\n\n## 5 . Melhorar a Argumentação e Diferenciação Comercial\n\n## ➡ O que 

In [8]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: about, products


In [9]:
for chunk in chunks:
    if 'products' or 'about' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='## 1 . Expansão Geográfica na América do Sul

Como vocês têm exclusividade , faz sentido expandir para outros países . Algumas ideias:

- ● Identificar os mercados mais promissores: Argentina , Chile e Peru têm setores fofortes de mineração e construção . Colômbia e Paraguai também podem ter demanda por reparo de maquinário pesado .
- ● Criar um canal de vendas internacional: Pode ser um site multilíngue ou um parceiro local para cada país .
- ● Aproveitar feiras internacionais: Participar de eventos na América do Sul voltados para mineração , construção e manutenção industrial .

## 2 . Reforçar a Geração de Leads e VeVendas Ativas' metadata={'source': '..\\preprocessing\\knowledge-base\\about\\Estratégia de Expansão - GPT.md', 'doc_type': 'about'}
_________
page_content='## 2 . Reforçar a Geração de Leads e VeVendas Ativas

- ● Inbound Marketing mais agressivo: Criar conteúdos educativos (artigos , vídeos e cases de sucesso) para atrair leads orgânicos .
- ● Campanhas m